In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import gffutils

In [ ]:
main_path = Path('..')
data_path = main_path / 'Data'
ref_path = data_path / 'GRCh38genome'
preprocessing_path = data_path / 'Preprocessing_LNDR_HNDR'
intersect_path = data_path / 'intersect_regions'
transcriptome_path = data_path / 'transcriptome'

In [ ]:
# fn = gffutils.example_filename(ref_path.absolute() / 'gencode.v47.basic.annotation.gff3')

In [ ]:
## One time operation

# db = gffutils.create_db(fn, dbfn=str(ref_path.resolve() / 'gencode.v47.basic.annotation.db'), force=True, keep_order=True, merge_strategy='merge', sort_attribute_values=True)
# print('done')

In [ ]:
db = gffutils.FeatureDB(str(ref_path.resolve() / 'gencode.v47.basic.annotation.db'))

In [ ]:
c = 0
transcript_to_gene = {}
for transcript in db.features_of_type('transcript'):
    if transcript.id not in transcript_to_gene:
        assert len(transcript['Parent']) == 1
        assert len(transcript['gene_id']) == 1
        assert transcript['gene_id'][0] == transcript['Parent'][0]
        transcript_to_gene[transcript.id] = transcript['Parent'][0]
    else:
        print(transcript)

In [ ]:
print(f'# of transcripts: {len(transcript_to_gene)}')

In [ ]:
# import json
# with open(ref_path / 'transcript_to_gene.json', 'w') as fout:
#     json.dump(transcript_to_gene, fout)

#### Some Checks

In [ ]:
import json

In [ ]:
with open(ref_path / 'transcript_to_gene.json', 'r') as file:
    transcript_to_gene = json.load(file)

transcript_to_gene_cleaned = {k.split('.')[0]:v.split('.')[0] for k,v in transcript_to_gene.items()}
# transcript_to_gene_cleaned = {k:v.split('.')[0] for k,v in transcript_to_gene.items()}

In [ ]:
len(transcript_to_gene), len(transcript_to_gene_cleaned)

In [ ]:
len(set(transcript_to_gene.values())), len(set(transcript_to_gene_cleaned.values())) ## 78724 genes

In [ ]:
df_expression = pd.read_csv(transcriptome_path / 'LTC_HepG2_MonoCal_FA.rep1.TPM.txt', sep='\t', names=['id', 'expression'], header=0)
df_expression['gene_id'] = df_expression['id'].str.split(':').str[0]
df_expression.head()

In [ ]:
df_expression['gene_id'].shape, df_expression['gene_id'].unique().shape

In [ ]:
from collections import defaultdict

In [ ]:
gene_to_transcript = defaultdict(list)
for t, g in transcript_to_gene_cleaned.items():
    gene_to_transcript[g].append(t)

In [ ]:
len(gene_to_transcript)

In [ ]:
mask_expressed = df_expression['expression'] > 0

def get_transcripts_list(series, map):
    t_list = []
    for g in series:
        t_list += map[g]

    return t_list

# transcript_expressed = [gene_to_transcript[gene] for gene in df_expression.loc[mask_expressed, 'gene_id'] if gene in gene_to_transcript]
# transcript_not_expressed = [gene_to_transcript[gene] for gene in df_expression.loc[~mask_expressed, 'gene_id'] if gene in gene_to_transcript]

transcript_expressed = get_transcripts_list(df_expression.loc[mask_expressed, 'gene_id'], gene_to_transcript)
transcript_not_expressed = get_transcripts_list(df_expression.loc[~mask_expressed, 'gene_id'], gene_to_transcript)

In [ ]:
len(transcript_expressed), len(transcript_not_expressed)

In [ ]:
transcript_expressed[:10]

In [ ]:
# 127625
total_n_transcripts = 0
for _id in df_expression['gene_id']:
    assert _id in gene_to_transcript
    total_n_transcripts += len(gene_to_transcript[_id])

total_n_transcripts

In [ ]:
GRCh38_annotations = []
with open(str(ref_path.resolve() / 'GRCh38.p14.annotations')) as fin:
    for line in fin:
        if line.startswith('ENST'):
            GRCh38_annotations.append(line.split()[0])
        else:
            print(line)

print(f'# of transcripts in ref used: {len(GRCh38_annotations)} | {len(set(GRCh38_annotations))}')
c = 0
for entry in GRCh38_annotations:
    # if entry in transcript_to_gene_cleaned:
    if entry.split('.')[0] in transcript_to_gene_cleaned:
        c += 1

print(f'# of transcripts present in map db: {c} of {len(GRCh38_annotations)}')

c = 0
for entry in GRCh38_annotations:
    # if entry in transcript_expressed:
    if entry.split('.')[0] in transcript_expressed:
        c += 1

print(f'# of transcripts expressed: {c} of {len(GRCh38_annotations)}')

c = 0
for entry in GRCh38_annotations:
    # if entry in transcript_not_expressed:
    if entry.split('.')[0] in transcript_not_expressed:
        c += 1

print(f'# of transcripts non-expressed: {c} of {len(GRCh38_annotations)}')

In [ ]:
def get_no_transcripts(path, label):
    transcripts = []
    with open(path) as fin:
        for line in fin:
            transcripts.append(line.split()[3])
    print(f'# of transcripts in {label} used: {len(transcripts)} | {len(set(transcripts))}')

    c = 0
    for entry in set(transcripts):
        # if entry in transcript_to_gene_cleaned:
        if entry.split('.')[0] in transcript_to_gene_cleaned:
            c += 1
    print(f'# of transcripts in {label} present in map db: {c} of {len(set(transcripts))}')

    c = 0
    for entry in set(transcripts):
        # if entry in transcript_expressed:
        if entry.split('.')[0] in transcript_expressed:
            c += 1
    print(f'# of transcripts in {label} expressed: {c} of {len(set(transcripts))}')

    c = 0
    for entry in set(transcripts):
        # if entry in transcript_not_expressed:
        if entry.split('.')[0] in transcript_not_expressed:
            c += 1
    print(f'# of transcripts in {label} non-expressed: {c} of {len(set(transcripts))}')
    print(' ')

get_no_transcripts(preprocessing_path / 'GRCh38.p14.promoter.sorted.bed', 'promoter')
get_no_transcripts(preprocessing_path / 'GRCh38.p14.intron.1.start.sorted.bed', 'intron.1.start')
get_no_transcripts(preprocessing_path / 'GRCh38.p14.intron.1.end.sorted.bed', 'intron.1.end')
get_no_transcripts(preprocessing_path / 'GRCh38.p14.intron.2.start.sorted.bed', 'intron.2.start')

In [ ]:
def get_no_transcripts(path, label):
    transcripts = []
    with open(path) as fin:
        for line in fin:
            transcripts.append(line.split()[3])
    print(f'# of transcripts in {label} used: {len(transcripts)} | {len(set(transcripts))}')

    c = 0
    for entry in set(transcripts):
        # if entry in transcript_to_gene_cleaned:
        if entry.split('.')[0] in transcript_to_gene_cleaned:
            c += 1
    print(f'# of transcripts present in map db: {c} of {len(set(transcripts))}')

    c = 0
    for entry in set(transcripts):
        # if entry in transcript_expressed:
        if entry.split('.')[0] in transcript_expressed:
            c += 1
    print(f'# of transcripts in {label} expressed: {c} of {len(set(transcripts))}')

    c = 0
    for entry in set(transcripts):
        # if entry in transcript_not_expressed:
        if entry.split('.')[0] in transcript_not_expressed:
            c += 1
    print(f'# of transcripts in {label} non-expressed: {c} of {len(set(transcripts))}')
    print(' ')

for region in ['promoter', 'intron.1.start', 'intron.1.end', 'intron.2.start']:
    get_no_transcripts(intersect_path / f'GCH.{region}.intersect.bed', f'GCH.{region}')
    get_no_transcripts(intersect_path / f'HCG.{region}.intersect.bed', f'HCG.{region}')

In [ ]:
def get_no_transcripts(path, label):
    transcripts = []
    with open(path) as fin:
        for line in fin:
            transcripts.append(line.split()[3])

    print(f'# of transcripts in {label} used: {len(transcripts)} | {len(set(transcripts))}')
    c = 0
    for entry in set(transcripts):
        if entry in transcript_to_gene:
            c += 1

    print(f'# of transcripts present in map db: {c} of {len(set(transcripts))}')
    print(' ')

for region in ['promoter', 'intron.1.start', 'intron.1.end', 'intron.2.start']:
    get_no_transcripts(intersect_path / f'NDR.{region}.intersect.bed', f'NDR.{region}')
    get_no_transcripts(intersect_path / f'NOR.{region}.intersect.bed', f'NOR.{region}')

In [ ]:
def get_no_transcripts(path, label):
    transcripts = []
    with open(path) as fin:
        for line in fin:
            transcripts.append(line.split()[3])

    print(f'# of transcripts in {label} used: {len(transcripts)} | {len(set(transcripts))}')
    c = 0
    for entry in set(transcripts):
        if entry in transcript_to_gene:
            c += 1

    print(f'# of transcripts present in map db: {c} of {len(set(transcripts))}')
    print(' ')

for region in ['promoter', 'intron.1.start', 'intron.1.end', 'intron.2.start']:
    get_no_transcripts(intersect_path / f'HCG.{region}.intersect.bed', f'HCG.{region}')

In [ ]:
def get_no_transcripts(path, label):
    transcripts = []
    with open(path) as fin:
        for line in fin:
            transcripts.append(line.split()[3])

    print(f'# of transcripts in {label} used: {len(transcripts)} | {len(set(transcripts))}')
    c = 0
    for entry in set(transcripts):
        if entry in transcript_to_gene:
            c += 1

    print(f'# of transcripts present in map db: {c} of {len(set(transcripts))}')
    print(' ')

for region in ['promoter', 'intron.1.start', 'intron.1.end', 'intron.2.start']:
    get_no_transcripts(intersect_path / f'{region}.NDR.HCG.intersect.bed', f'NDR.HCG.{region}')
    get_no_transcripts(intersect_path / f'{region}.NOR.HCG.intersect.bed', f'NOR.HCG.{region}')

#### Read trancriptome data

In [ ]:
df_expression = pd.read_csv(transcriptome_path / 'LTC_HepG2_MonoCal_FA.rep1.TPM.txt', sep='\t', names=['id', 'expression'], header=0)

In [ ]:
df_expression['gene_id'] = df_expression['id'].str.split(':').str[0]

In [ ]:
df_expression.head()

In [ ]:
df_expression['expression'].hist()
plt.yscale('log')

In [ ]:
df_expression.loc[df_expression['expression'] < 100, 'expression'].hist()
plt.yscale('log')

In [ ]:
df_expression['expression'].quantile(0.20), df_expression['expression'].quantile(0.80), df_expression['expression'].quantile(0.85)

In [ ]:
df_expression.shape

In [ ]:
genes = []
for gene_id in transcript_to_gene.values():
    genes.append(gene_id.split('.')[0])

c = 0
missing = []
for gene_id in df_expression['gene_id']:
    if gene_id in genes:
        c += 1
    else:
        missing.append(gene_id)
        
print(f'genes from expression data present in map {c}')
print(f"genes missing {df_expression['gene_id'].shape[0] - c} | {len(missing)}")

In [ ]:
df_expression.loc[df_expression['gene_id'].isin(missing), 'expression'].hist()
plt.yscale('log')

In [ ]:
transcript_to_gene